# Module 2: Preprocessing, indexing and approximate matching

## Exact matching: online algorithms

Online algorithms don't process the text `t`, but some process the pattern `p`.

### Naive exact matching

* compares pattern `p` to every offset in the text `t`

* lots of comparisons = very slow an inefficient

### Boyer-Moore exact matching

* an efficient string-searching algorithm
* preprocesses the pattern **P** (the string being searched for), but not the text **T** (the string being searched in)
* well-suited for applications in which the pattern is much shorter than the text or where it persists across multiple searches
* uses information gathered during the preprocess step to skip sections of the text
* for each alignment the last character in P is compared first

#### Bad character rule

* considers the character in T at which the comparison process failed (assuming such a failure occurred)
* upon missmatch, skip alignments until the missmatch becomes a match or pattern moves past missmatched character.

E.g. 

alignment 1: missmatch
```
T: GCTTCTGCTACCTTTT
P: CCTTTTGC
       x✓✓✓
```

alignment 2: still a missmatch - skip
```
T: GCTTCTGCTACCTTTT
P:  CCTTTTGC
       x
```

alignment 3: still a missmatch - skip
```
T: GCTTCTGCTACCTTTT
P:   CCTTTTGC
       x
```

alignment 4: match - do not skip
```
T: GCTTCTGCTACCTTTT
P:    CCTTTTGC
       ✓
```

#### Good suffix rule

???

## Exact matching: index assisted offline algorithms

Offline algorithms do process the text `t`.

### K-mer indexing

* `k`-mer = substring of `t` of length `k`

* For every `k`-mer in `t`, record the index (aka the offset) where it occurs in `t`

* For a given pattern `p` can find all the offsets where `p` occurs in `t` using the `k`-mer index

  * e.g. take the first `k`-mer in `p` and look it up in the index

  * e.g. if a match is found at offset `i` then we know that `p[:k] == t[i:i+k]`
  
  * next need to verify that the rest of `p` also matches `t` - i.e. we need to check that `p[k:] == t[i+k:i+len(p)]`

#### Implementation

I.e. how to actually find the offset in the index for a given k-mer.

* binary search (e.g. using bisect module)

* hash table (e.g. dictionary)

#### Variations

* build the index using every nth k-mer from `t`
  * e.g. use every k-mer with and even offset
    * need to query the index twice - e.g. using k-mers in `p` with offsets `i` and `i+1`
  * e.g. use every 3rd k-mer
    * need to query the index 3 times - e.g using k-mers in `p` with offsets `i`, `i+2` and `i+4`

* build the index using subsequences instead of substrings
  * a subsequence of S = a string of characters that also occurs in S in the same order, but not necessarily consecutively.
    * e.g. `'ABC'` is a subsequence of `'xxxAxxxBxxxCxxx'`
  * a substring is a special case of a subsequence where the characters are consecutive
    * therefore all strings are subsequences
    * but not all subsequences are substrings!
  * this method tends to increase the specificity of the filter provided by the index compared to the method that uses substrings - i.e. index hits lead to successful verifications more of the time (is this really true?)

### Other kinds of indexing using in genomics

* suffix tree

* suffix array

* fm index

## Approximate matching

Good reasons to expect differences between the read and the reference genome:

* sequencing errors

* reference genome might be different from the genome being studied

### Types of differences between strings

* missmatch - aka substitution

* insertion

* deletion

### Measurements of distance between strings

#### Hamming distance

* minimum number of substitutions needed to turn one string into the other

* strings need to be the same length

#### Edit distance (aka Levenstein distance)

* minimum number of substitutions / insertions / deletions needed to turn one string into the other

* strings don't need to be the same length

In [1]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

def naive_hamming(p, t, max_distance):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        n_mis_match = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                n_mis_match += 1
                if n_mis_match > max_distance:
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

### Pigeonhole principle

Allows us to apply exact matching algorithms to approximate matching problems

* E.g. if we want to allow a maximum of 1 edit when matching `p` to `t`
    * if we split `p` into 2 partitions then at least one of those partitions must match `t` exactly

* E.g. if we want to allow a maximum of `n` edits when matching `p` to `t`
    * if we split `p` into `n+1` partitions then at least one of those partitions must match `t` exactly


In [63]:
import math

def partition_string1(x, n):
    assert n > 0
    assert n <= len(x)
    partitions = []
    partition_size = round(len(x) / n)
    for i in range(n):
        start = i * partition_size
        end = min((i+1) * partition_size, len(x))
        partitions.append(x[start:end])
    return partitions

def partition_string2(x, n):
    assert n > 0
    assert n <= len(x)
    partitions = []
    partition_size = math.ceil(len(x) / n)
    for i in range(n):
        start = i * partition_size
        end = min((i+1) * partition_size, len(x))
        partitions.append(x[start:end])
    return partitions

import textwrap

def partition_string3(x, n):
    assert n > 0
    assert n <= len(x)
    partition_size = round(len(x) / n)
    return textwrap.wrap(x, partition_size)

def partition_string4(x, n):
    assert n > 0
    assert n <= len(x)
    partition_size = math.ceil(len(x) / n)
    return textwrap.wrap(x, partition_size)

In [68]:
x = '12345678901234567890'
[partition_string1(x, i) for i in [1,2,3,4,5,6,7,8,9,10]]

[['12345678901234567890'],
 ['1234567890', '1234567890'],
 ['1234567', '8901234', '567890'],
 ['12345', '67890', '12345', '67890'],
 ['1234', '5678', '9012', '3456', '7890'],
 ['123', '456', '789', '012', '345', '678'],
 ['123', '456', '789', '012', '345', '678', '90'],
 ['12', '34', '56', '78', '90', '12', '34', '56'],
 ['12', '34', '56', '78', '90', '12', '34', '56', '78'],
 ['12', '34', '56', '78', '90', '12', '34', '56', '78', '90']]

In [69]:
[partition_string2(x, i) for i in [1,2,3,4,5,6,7,8,9,10]]

[['12345678901234567890'],
 ['1234567890', '1234567890'],
 ['1234567', '8901234', '567890'],
 ['12345', '67890', '12345', '67890'],
 ['1234', '5678', '9012', '3456', '7890'],
 ['1234', '5678', '9012', '3456', '7890', ''],
 ['123', '456', '789', '012', '345', '678', '90'],
 ['123', '456', '789', '012', '345', '678', '90', ''],
 ['123', '456', '789', '012', '345', '678', '90', '', ''],
 ['12', '34', '56', '78', '90', '12', '34', '56', '78', '90']]

In [70]:
[partition_string3(x, i) for i in [1,2,3,4,5,6,7,8,9,10]]

[['12345678901234567890'],
 ['1234567890', '1234567890'],
 ['1234567', '8901234', '567890'],
 ['12345', '67890', '12345', '67890'],
 ['1234', '5678', '9012', '3456', '7890'],
 ['123', '456', '789', '012', '345', '678', '90'],
 ['123', '456', '789', '012', '345', '678', '90'],
 ['12', '34', '56', '78', '90', '12', '34', '56', '78', '90'],
 ['12', '34', '56', '78', '90', '12', '34', '56', '78', '90'],
 ['12', '34', '56', '78', '90', '12', '34', '56', '78', '90']]

In [71]:
[partition_string4(x, i) for i in [1,2,3,4,5,6,7,8,9,10]]

[['12345678901234567890'],
 ['1234567890', '1234567890'],
 ['1234567', '8901234', '567890'],
 ['12345', '67890', '12345', '67890'],
 ['1234', '5678', '9012', '3456', '7890'],
 ['1234', '5678', '9012', '3456', '7890'],
 ['123', '456', '789', '012', '345', '678', '90'],
 ['123', '456', '789', '012', '345', '678', '90'],
 ['123', '456', '789', '012', '345', '678', '90'],
 ['12', '34', '56', '78', '90', '12', '34', '56', '78', '90']]

In [ ]:
def approximate_match(p, t, n):
    # divide p into n+1 segments - at least 1 must match perfectly against t
    segment_length = round(len(p) / (n+1))
    all_matches = set()
    for i in range(n+1):
        start = i * segment_length
        end = min((i+1) * segment_length, len(p))